# Подготовка модели

## Установка предобученной

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.4 MB/s eta 0:00:00


In [15]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Работаем с русскоязычной GPT от Сбера.
# Ниже команды для загрузки и инициализации модели и токенизатора.
model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [16]:
PYTORCH_CUDA_ALLOC_CONF = "max_split_size_mb:512"

Проверяем, как работает предуобученная

In [ ]:
# prompt engineering for QA
text = "Вопрос: 'Сколько будет 2+2?'\nОтвет:" 
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
out = model.generate(input_ids, do_sample=False) 

generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Вопрос: 'Сколько будет 2+2?'
Ответ: '2+2=4'


# Составляем данные, на которых будет учиться модель

In [1]:
import tqdm
from bs4 import BeautifulSoup
import requests
import re

In [2]:
session = requests.session()
adapter = requests.adapters.HTTPAdapter(
    pool_connections=10, # Количество пулов соединений
    pool_maxsize=10 # Размер каждого пула
)
session.mount('https://', adapter)
response = session.get('https://killpls.me')

In [3]:
def page_to_dict(page):
    texts_list = []
    texts = page.findAll('div', class_='col-xs-12',style='margin:0.5em 0;line-height:1.785em')
    id_date_tags = page.findAll('div', class_='col-xs-6')
    for ct in range(len(texts)):
        temp_dict = {}
        # id_text
        text = texts[ct].text.replace('\n','').replace('\r','')
        text = texts[ct].text.replace('\t','')
        text = text.replace('        ','').replace('    ','')
        # tags
        tags = re.findall(r'(?<=>)[а-я]*(?=</a>)', str(id_date_tags[ct*2+1]))
        temp_dict['tags'] = tags
        temp_dict['text'] = text
        texts_list.append(temp_dict)
    return texts_list

In [ ]:
req = session.get(f'https://killpls.me/page/1000')
page = BeautifulSoup(req.text, 'html.parser')
page_to_dict(page)

In [10]:
list_all = []
errors = []
for count_number in tqdm.trange(100,200):
    req = session.get(f'https://killpls.me/page/{str(count_number)}')
    try:
        page = BeautifulSoup(req.text, 'html.parser')
        list_all.append(page_to_dict(page))
    except:
        errors.append(count_number)

100%|██████████| 100/100 [00:52<00:00,  1.90it/s]


In [11]:
list_out = []
count = {}
for item_list in list_all:
    for item in item_list:
        if item['text'] != '18+':
            list_out.append(item["text"])
        

In [12]:
with open('/content/corpus_kmp.txt', 'w', encoding='utf-8') as f:
    #string_data = ''
    #print('[',file=f)
    for item in list_out:
        if '18+' not in item:
            item = item.replace('\n','')
            item = re.sub(r'\.{2,5}', '.',item)
            item = re.sub(r'(?<=[А-я])\.(?=[А-я])', '. ',item)
            print(item,file=f)
    #print(']',file=f)

# Файнтюнинг

In [13]:
import torch
torch.cuda.empty_cache()

In [20]:
from transformers import TextDataset, DataCollatorForLanguageModeling

train_path = '/content/corpus_kmp.txt'
# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=8)
  
# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                mlm=False)

In [21]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned", # The output directory
    overwrite_output_dir=True, # Overwrite the content of the output dir
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    warmup_steps=10, # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16, # to make "virtual" batch size larger
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
)

In [22]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=245, training_loss=4.409580277423469, metrics={'train_runtime': 964.1435, 'train_samples_per_second': 65.685, 'train_steps_per_second': 0.254, 'total_flos': 2046275715465216.0, 'train_loss': 4.409580277423469, 'epoch': 4.95})

In [23]:
torch.save(model, '/content/drive/MyDrive/proga_temp/model_new.pt')

In [7]:
from transformers import WEIGHTS_NAME, CONFIG_NAME
#output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
#output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model.state_dict(), f'/content/drive/MyDrive/proga_temp/{WEIGHTS_NAME}')
model.config.to_json_file(f'/content/drive/MyDrive/proga_temp/{CONFIG_NAME}')
tokenizer.save_pretrained(f'/content/drive/MyDrive/proga_temp/')

('/content/drive/MyDrive/proga_temp/tokenizer_config.json',
 '/content/drive/MyDrive/proga_temp/special_tokens_map.json',
 '/content/drive/MyDrive/proga_temp/vocab.json',
 '/content/drive/MyDrive/proga_temp/merges.txt',
 '/content/drive/MyDrive/proga_temp/added_tokens.json')

In [5]:
text = "Я очень хочу спать"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]

print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Я очень хочу спать. Не могу уснуть, постоянно ворочаюсь. В голову лезет всякая фигня. ПМП. Захожу в комнату, а там на диване лежит моя лучшая подруга. Она в одной ночнушке, в другой - в трусах. И я не знаю, что делать. У неё на коленях куча денег, и она говорит, что это всё она заработала и теперь она может тратить их на всякую ерунду. Мне дико стыдно, что я не могу дать ей ни копейки. А она заявляет, что это её зарплата. ПМП. В итоге я в бешенстве от всего этого, но в то же время рад, что она меня не бросила. ПМП. ПМП. ПМП. ПМП. Блядь. ПМП. БЛЯТЬ. ПМП. ПМП. ПМП. ПМП. ПМП. ПМП. ПМП. ПМП. ПМП. ПМП. ПМП. ПМП. ПМП. ПМП
